In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
import pickle
import requests
import numpy as np
import pandas as pd
import time
import csv
import re
import json
import xlrd
import os
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime, date, timedelta
import nltk
#from flatten_json import flatten
regex = re.compile(r"\[|\]|<", re.IGNORECASE)

import time

In [2]:
%cd /home/stu/code/dans

/home/stu/code/dans


In [1]:

### Dans
driver = webdriver.Firefox(executable_path=r'/usr/local/bin/geckodriver')
path = "https://www.danmurphys.com.au/red-wine/all"
driver.get(path)   

file = "scraped_" + time.strftime("%Y%m%d") + ".csv"


with open(file, "w") as csvFile:
    fieldnames = ['URL']
    writer = csv.DictWriter(csvFile, fieldnames=fieldnames)
    writer.writeheader()
    for item in range(1,2):
        path = "https://www.danmurphys.com.au/red-wine/all?page=" + str(item)
        driver.implicitly_wait(2)
        driver.get(path)
        elems = driver.find_elements_by_xpath("//a[@href]")
        for elem in elems:
            writer.writerow({'URL': elem.get_attribute("href")})




KeyboardInterrupt: 

In [10]:
import time

print(file)

scraped_20210809.csv


,URL
count,1228
unique,546
top,https://www.danmurphys.com.au/liquor-library
freq,22


In [4]:
import glob

path = r'/home/stu/code/dans' # use your path
all_files = glob.glob(path + "/scraped*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [5]:
wines = frame
# And now I'll filter out the product linkes
wines = wines[wines.URL.str.contains('https://www.danmurphys.com.au/product')]

wines = wines.replace(r'https://www.danmurphys.com.au/product/DM_', r'', regex=True)
wines = wines.replace(r'/.*$', r'', regex=True)
wines = wines.drop_duplicates()


In [17]:
#wines
#mylist = df['Cleaned'].tolist()
mylist = wines['URL'].tolist()

#ylist = mylist[0:100]
result = []

for wine in mylist:
    url = "https://api.danmurphys.com.au/apis/ui/Product/" + str(wine)
    r = requests.get(url).json()
    try:
        rtoo = pd.json_normalize(r["Products"])
        result.append(rtoo)
    except:
        print("No product found")

my_df = pd.concat(result)
file_res = "API_results_" + time.strftime("%Y%m%d") + ".csv"
my_df.to_csv(file_res)
#pd.json_normalize(flatten(r))




In [123]:

my_df = pd.read_csv("API_results_20210812.csv")

## Categories
#my_df["Categories"]
# Willnjust keep 2 levels.
my_df["Categories"] = my_df["Categories"].map(eval, na_action='ignore')
new_df = my_df["Categories"].apply(pd.Series)
my_df["Categories"] = new_df[0].apply(pd.Series).UrlFriendlyName
my_df["Sub_Categories"] = new_df[1].apply(pd.Series).UrlFriendlyName



In [124]:

## Reviews
my_df["Reviews"] = my_df["Reviews"].map(eval,  na_action='ignore')
# Try with first 2 reviews
new_df = my_df["Reviews"].apply(pd.Series)
# First
my_df["Review1_auth"] = new_df[0].apply(pd.Series).author.apply(pd.Series).Value
my_df["Review1_authorcontent"] = new_df[0].apply(pd.Series).authorcontent.apply(pd.Series).Value
my_df["Review1_points"] = new_df[0].apply(pd.Series).points.apply(pd.Series).Value
my_df["Review1_source"] = new_df[0].apply(pd.Series).source.apply(pd.Series).Value
my_df["Review1_text"] = new_df[0].apply(pd.Series).text.apply(pd.Series).Value
my_df["Review1_vintage"] = new_df[0].apply(pd.Series).vintage.apply(pd.Series).Value
# Second
my_df["Review1_auth"] = new_df[1].apply(pd.Series).author.apply(pd.Series).Value
my_df["Review1_authorcontent"] = new_df[1].apply(pd.Series).authorcontent.apply(pd.Series).Value
my_df["Review1_points"] = new_df[1].apply(pd.Series).points.apply(pd.Series).Value
my_df["Review1_source"] = new_df[1].apply(pd.Series).source.apply(pd.Series).Value
my_df["Review1_text"] = new_df[1].apply(pd.Series).text.apply(pd.Series).Value
my_df["Review1_vintage"] = new_df[1].apply(pd.Series).vintage.apply(pd.Series).Value



In [125]:
# Additional details
my_df["AdditionalDetails"] = my_df["AdditionalDetails"].map(eval,  na_action='ignore')

In [126]:
#my_df.to_csv("pre-norm.csv")

In [127]:
import copy
full_df = copy.deepcopy(my_df)

In [128]:
full_df.to_csv("full.csv")

In [129]:
# Can't use nested lists of JSON objects in pd.json_normalize
my_df = my_df.explode(column="AdditionalDetails").reset_index(drop=True)

In [46]:
#pd.json_normalize(my_df['AdditionalDetails']).pivot(columns='Name')['Value']

In [47]:
# pd.json_normalize expects a list of JSON objects not a DataFrame
#my_df = pd.concat([my_df, pd.json_normalize(my_df['AdditionalDetails'])], axis=1) 

In [131]:
add_df = pd.DataFrame(pd.json_normalize(my_df["AdditionalDetails"]))

In [132]:
del add_df["DisplayName"]

In [133]:
df = pd.concat([my_df,add_df],axis=1)

In [134]:
df = df.pivot(index='Stockcode',columns='Name', values='Value')

In [135]:
df.to_csv("mydf.csv")
df = pd.read_csv("mydf.csv")

In [73]:
#newdf =  pd.concat([full_df,df],axis=1)

In [111]:
#full_df['Stockcode'] = df['Stockcode'].astype(str)

In [112]:
#newdf = full_df.set_index('Stockcode').join(df.set_index('Stockcode'), lsuffix='_full', rsuffix='_new')

In [137]:
newdf = pd.merge(full_df, df, on='Stockcode')


In [138]:
newdf.to_csv("newdf.csv")
newdf = pd.read_csv("newdf.csv")

In [139]:
list(newdf)

['Unnamed: 0',
 'Unnamed: 0.1',
 'RecommendedProducts',
 'ProductsInSameOffer',
 'Categories',
 'BackorderMessage',
 'Reviews',
 'DeliveryOptionsInfo',
 'SavedLists',
 'Stockcode',
 'Description',
 'SmallImageFile',
 'MediumImageFile',
 'LargeImageFile',
 'IsOnSpecial',
 'IsMemberSpecial',
 'IsEdrSpecial',
 'QuantityInTrolley',
 'Unit',
 'MinimumQuantity',
 'IsInTrolley',
 'Source',
 'SupplyLimit',
 'PackageSize',
 'IsForCollection',
 'IsForDelivery',
 'IsPurchasable',
 'IsFindMeAvailable',
 'ShouldShowFindMeCta',
 'ProductTags',
 'ProductSashes',
 'UniqueSellingProposition',
 'AgeRestricted',
 'DisplayQuantity',
 'RichDescription',
 'IsFeaturedTag',
 'OverallRating',
 'NumberOfReviews',
 'IsNew',
 'ImageVariants',
 'AdditionalDetails',
 'ParentStockCode',
 'StockOnHand',
 'BackorderStockOnHand',
 'IsPreSale',
 'IsComingSoon',
 'UrlFriendlyName',
 'AvailablePackTypes',
 'IsInWishList',
 'IsInDefaultList',
 'IsDeliveryOnly',
 'GroupedDetails.image',
 'GroupedDetails.workflow',
 'PickupS

In [ ]:
['awardwinner',
 'countryoforigin',
 'glutenfree',
 'preservativefree',
 'standarddrinks',
 'varietal',
 'webalcoholpercentage',
 'webbottleclosure',
  'webdescriptionshort',
 'webfoodmatch',
 'webisorganic',
 'webisvegan',
 'webliquorsize',
 'webpacksizecase',
 'webproductname',
 'webproductsale',
 'webproducttype',
 'webpromomdmmessage',
 'webpromomessage',
 'webpromomessageenddt',
 'webpromomessagestartdt',
 'webpromosecondmesgenddt',
 'webpromosecondmesgstartdt',
 'webpromosecondmessage',
 'webpromotionalbundle',
 'webregionoforigin',
 'webstateoforigin',
 'webtitle',
 'webtotalreviewcount',
 'webvideourl',
 'webvintagecurrent',
 'webvintagenote',
 'webwinebody',
 'webwinemaker',
 'webwinestyle']

In [ ]:
Text to TFIDF
'webdescriptionshort',


In [141]:
gives = newdf[['Stockcode','webproductname','Prices.singleprice.Value','Prices.promoprice.Value','Prices.promoprice.BeforePromotion','Prices.promoprice.AfterPromotion']]

In [103]:
gives

,Stockcode,webproductname,Prices.singleprice.Value_full,Prices.promoprice.Value_full,Prices.promoprice.BeforePromotion_full,Prices.promoprice.AfterPromotion_full
0,130687,NaN,14.99,NaN,NaN,NaN
1,132881,NaN,49.99,NaN,NaN,NaN
2,830650,NaN,13.95,NaN,NaN,NaN
3,796067,NaN,23.99,NaN,NaN,NaN
4,ER_1000007337_OHFRSHZ,NaN,219.95,NaN,NaN,NaN
...,...,...,...,...,...,...
2734,765272,NaN,119.99,NaN,NaN,NaN
2735,126021,NaN,119.99,NaN,NaN,NaN
2736,143119,NaN,17.90,NaN,NaN,NaN
2737,378507,NaN,122.99,NaN,NaN,NaN


In [142]:
gives = gives[gives.webproductname.notnull()]

In [143]:
gives

,Stockcode,webproductname,Prices.singleprice.Value,Prices.promoprice.Value,Prices.promoprice.BeforePromotion,Prices.promoprice.AfterPromotion
3,796067,Barossa Valley Estate Shiraz,23.99,NaN,NaN,NaN
7,144469,Pepperjack Shiraz,17.95,102.10,102.30,102.10
8,904960,Penfolds Koonunga Hill Shiraz,14.99,67.70,85.80,67.70
11,20345,Wynns Shiraz,13.95,70.05,83.70,70.05
12,322002,Cat Amongst The Pigeons Shiraz,15.99,13.00,15.99,13.00
...,...,...,...,...,...,...
2729,907709,Climbing Shiraz,17.99,15.00,17.99,15.00
2731,760953,Warburn Rumours Cabernet Merlot,5.49,NaN,NaN,NaN
2732,431873,Pegasus Bay Prima Donna Pinot Noir,124.99,NaN,NaN,NaN
2734,765272,Howard Park Abrcrombie Cabernet Sauvignon 2009,119.99,NaN,NaN,NaN


In [144]:
gives = gives[gives['Stockcode'].str.contains("MYSTERY")]

In [145]:
gives

,Stockcode,webproductname,Prices.singleprice.Value,Prices.promoprice.Value,Prices.promoprice.BeforePromotion,Prices.promoprice.AfterPromotion
1248,MYSTERY202,Under Wraps Mornington Peninsula Pinot Noir 2018,44.99,15.0,44.99,15.0
1251,MYSTERY187,Under Wraps McLaren Vale Cabernet 2015,34.99,16.0,34.99,16.0
1274,MYSTERY437,Alkoomi Blackbutt Cabernet Merlot Cabernet Franc,59.99,30.0,59.99,30.0
1297,MYSTERY261,Known Pleasures McLaren Vale Shiraz 2017,59.99,20.0,59.99,20.0
1308,MYSTERY82,Under Wrapss McLaren Vale Shiraz 2015,21.99,12.0,21.99,12.0
1382,MYSTERY378,Peos Estate Cab Sauv,22.99,15.0,22.99,15.0
1432,MYSTERY51,Under Wraps Margaret River Cabernet Sauvignon ...,74.99,30.0,74.99,30.0
1499,MYSTERY44,Under Wraps Single Vineyard McLaren Vale Shira...,84.99,35.0,84.99,35.0
1656,MYSTERY540,Jim Barry The Benbournie Cabernet Sauvignon 2014,35.88,NaN,NaN,NaN
1664,MYSTERY459,Forest Hill Block 5 Caberent Sauvignon,30.00,NaN,NaN,NaN


In [146]:
gives = gives[~gives["webproductname"].str.contains("Wraps")]

In [147]:
gives

,Stockcode,webproductname,Prices.singleprice.Value,Prices.promoprice.Value,Prices.promoprice.BeforePromotion,Prices.promoprice.AfterPromotion
1274,MYSTERY437,Alkoomi Blackbutt Cabernet Merlot Cabernet Franc,59.99,30.0,59.99,30.0
1297,MYSTERY261,Known Pleasures McLaren Vale Shiraz 2017,59.99,20.0,59.99,20.0
1382,MYSTERY378,Peos Estate Cab Sauv,22.99,15.0,22.99,15.0
1656,MYSTERY540,Jim Barry The Benbournie Cabernet Sauvignon 2014,35.88,NaN,NaN,NaN
1664,MYSTERY459,Forest Hill Block 5 Caberent Sauvignon,30.00,NaN,NaN,NaN
1676,MYSTERY242,Gibson My Darling Merlot 2014,24.99,12.0,24.99,12.0
2071,MYSTERY594,Feathered Friends Heathcote Shiraz Cabernet 2015,25.00,15.0,25.00,15.0
2506,MYSTERY411,Craggy Range Le Sol Syrah 2010,155.99,79.9,155.99,79.9
2590,MYSTERY460,Forest Hill Block 9 Shiraz,59.99,26.9,59.99,26.9
2711,MYSTERY513,*,39.99,15.0,39.99,15.0


In [30]:
df[df['Stockcode'].str.contains("Gibson My Darling Merlot 2014")]


,Stockcode,awardwinner,brewery,categoryleafnodeid,corkscoreeligible,countryoforigin,dm_stockcode,glutenfree,image1,image2,...,webregionoforigin,webstateoforigin,webtitle,webtotalreviewcount,webvideourl,webvintagecurrent,webvintagenote,webwinebody,webwinemaker,webwinestyle


In [ ]:
Free to see
Mystery "webproductname"

In [14]:




#check = pd.json_normalize(my_df["AdditionalDetails"]).pivot(columns='Name')

#check.to_csv("check.csv")

#check['Value']
#my_df["AdditionalDetails"].apply(pd.Series.explode)

########### Would likely want ot save to DB at this tome/.

#In excel: =TEXTJOIN("','",TRUE,A1:CF1)
#keep = ['Categories','BackorderMessage','varietal','DeliveryOptionsInfo','SavedLists','Stockcode','PackageSize','RichDescription','StockOnHand','UrlFriendlyName','IsDeliveryOnly','Prices.inanysixprice.Value','Prices.caseprice.Value','Prices.singleprice.Value','Inventory.availableinventoryqty','Prices.promoprice.Message','Prices.promoprice.Value','Prices.promoprice.PreText','Prices.promoprice.BeforePromotion','Prices.promoprice.AfterPromotion','Sub_Categories','Review1_auth','Review1_authorcontent','Review1_source','Review1_points','Review1_text','Review1_vintage','awardwinner','brewery','corkscoreeligible','countryoforigin','dm_stockcode','image1','image2','standarddrinks','webalcoholpercentage','webaverageproductrating','webbadgescollection','webbottleclosure','webcountryoforigin','webdescriptionshort','webdsvflag','webfoodmatch','webisvegan','weblangtonsclassification','webliquorsize','webmaincategory','webmaxquantity','webminquantity','webpacksizecase','webpacktype','webpresaleenddate','webpresaleflag','webpresalemarketlaunchdate','webpresalemdmmaxqtylimit','webpresaleorderreleasedate','webpresalestartdate','webpresaleusermaxqtylimit','webproductcanbechilled','webproductname','webproductsale','webproducttype','webpromomdmmessage','webpromomessage','webpromomessageenddt','webpromomessagestartdt','webpromosecondmesgenddt','webpromosecondmesgstartdt','webpromosecondmessage','webpromotionalbundle','webregionoforigin','webstateoforigin','webtotalreviewcount','webvideourl','webvintagecurrent','webvintagenote','webwinebody','webwinemaker','webwinestyle']
#keep = ['Categories','BackorderMessage','Description','DeliveryOptionsInfo','SavedLists','Stockcode','PackageSize','StockOnHand','UrlFriendlyName','IsDeliveryOnly','Prices.inanysixprice.Value','Prices.caseprice.Value','Prices.singleprice.Value','Inventory.availableinventoryqty','Prices.promoprice.Message','Prices.promoprice.Value','Prices.promoprice.PreText','Prices.promoprice.BeforePromotion','Prices.promoprice.AfterPromotion','Sub_Categories','Review1_auth','Review1_authorcontent','Review1_source','Review1_points','Review1_text','Review1_vintage','awardwinner','brewery','corkscoreeligible','countryoforigin','dm_stockcode','image1','image2','standarddrinks','webalcoholpercentage','webaverageproductrating','webbadgescollection','webbottleclosure','webcountryoforigin','webdescriptionshort','webdsvflag','webfoodmatch','webisvegan','weblangtonsclassification','webliquorsize','webmaincategory','webmaxquantity','webminquantity','webpacksizecase','webpacktype','webpresaleenddate','webpresaleflag','webpresalemarketlaunchdate','webpresalemdmmaxqtylimit','webpresaleorderreleasedate','webpresalestartdate','webpresaleusermaxqtylimit','webproductcanbechilled','webproductname','webproductsale','webproducttype','webpromomdmmessage','webpromomessage','webpromomessageenddt','webpromomessagestartdt','webpromosecondmesgenddt','webpromosecondmesgstartdt','webpromosecondmessage','webpromotionalbundle','webregionoforigin','webstateoforigin','webtotalreviewcount','webvideourl','webvintagecurrent','webvintagenote','webwinebody','webwinemaker','webwinestyle']
# Timmed version
keep = ['Description','Stockcode','PackageSize','Prices.inanysixprice.Value','Prices.caseprice.Value','Prices.singleprice.Value','Prices.promoprice.Message','Prices.promoprice.Value','Prices.promoprice.PreText','Sub_Categories','Review1_auth','Review1_points','Review1_vintage','awardwinner','countryoforigin','standarddrinks','webalcoholpercentage','webaverageproductrating','webbadgescollection','webbottleclosure','webcountryoforigin','webdescriptionshort','webdsvflag','webfoodmatch','webisvegan','weblangtonsclassification','webliquorsize','webmaincategory','webpacksizecase','webpacktype','webregionoforigin','webstateoforigin','webvintagecurrent','webvintagenote','webwinebody','webwinestyle']
# Now Keep them.
kept = df[keep]
#kept.to_csv('fri_big_run_kept.csv')
kept.reset_index(drop=True, inplace=True)

KeyError: "['Prices.promoprice.PreText', 'Prices.promoprice.Message', 'Prices.caseprice.Value', 'Prices.singleprice.Value', 'Prices.promoprice.Value', 'Prices.inanysixprice.Value', 'PackageSize', 'Review1_points', 'Review1_vintage', 'Stockcode', 'Review1_auth', 'Description', 'Sub_Categories'] not in index"

In [ ]:

#myst.to_csv('fri_big_run_kept_myst.csv')
#known.to_csv('fri_big_run_kept_known.csv')
#myst = pd.read_csv("fri_big_run_kept_myst.csv")
#known = pd.read_csv("fri_big_run_kept_known.csv")

## Adding in tfidf for description, and then dropping the columns

from sklearn.feature_extraction.text import TfidfVectorizer

v = TfidfVectorizer(ngram_range=(2, 3))
x = v.fit_transform(kept['Description'])
# Convert to datafram
df1 = pd.DataFrame(x.toarray(), columns=v.get_feature_names())

res = pd.concat([kept, df1], axis=1)

y = v.fit_transform(kept['webdescriptionshort'].values.astype('U'))
df2 = pd.DataFrame(y.toarray(), columns=v.get_feature_names())

new_kept = pd.concat([res, df2], axis=1)

new_kept = new_kept.drop("Description", axis=1)

new_kept = new_kept.drop("webdescriptionshort", axis=1)

known = new_kept
known.reset_index(drop=True, inplace=True)
#### ONE HOT ENCODED
##### First i split into numeric and nominal. OHE the nominal
exclude_col = known.select_dtypes(include=np.number).columns.tolist() + ["Stockcode"]
my_df_num = known[exclude_col]
my_df_cat = known.drop(exclude_col, axis=1)
#my_df_cat.to_csv("FIN.csv")
my_df_cat_ohe = pd.get_dummies(my_df_cat)
my_df_ohe = pd.concat([my_df_num,my_df_cat_ohe], axis=1)
my_df_ohe = my_df_ohe.fillna(0)
my_df_ohe = my_df_ohe.replace(np.nan, 0)

# Drop duplicates #TODO check whats better to keep
my_df_ohe = my_df_ohe.loc[:,~my_df_ohe.columns.duplicated()]
# Clean up names
my_df_ohe.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in my_df_ohe.columns.values]


myst  = my_df_ohe[my_df_ohe.Stockcode.str.contains("MYSTERY")]
known  = my_df_ohe[~my_df_ohe.Stockcode.str.contains("MYSTERY")]

myst_nn = myst.drop("Stockcode", axis=1)
known_nn = known.drop("Stockcode", axis=1)


#my_df_ohe.to_csv("OHE.csv")
#myst_nn = myst_nn.reset_index()
#myst = myst.reset_index()

#known_nn = known_nn.reset_index()
#known = known.reset_index()

##### Now myst
#my_df_num_m = myst[exclude_col]
#my_df_cat_m = myst.drop(exclude_col, axis=1)
#my_df_cat.to_csv("FIN.csv")
#my_df_cat_ohe_m = pd.get_dummies(my_df_cat_m)
#my_df_ohe_m = pd.concat([my_df_num_m,my_df_cat_ohe_m], axis=1)
#my_df_ohe_m = my_df_ohe_m.drop("Stockcode", axis=1)
#my_df_ohe_m = my_df_ohe_m.fillna(0)
#my_df_ohe_m = my_df_ohe_m.replace(np.nan, 0)


from sklearn.neighbors import NearestNeighbors
# Create the k-NN model using k=5
nn_abs = NearestNeighbors(n_neighbors=5, algorithm='auto')



# Checkif we've got NsNS

#
#wines_corr =  my_df_ohe_d.corr(method = "pearson")
#
nn_abs.fit(known_nn)

distance, matches = nn_abs.kneighbors(myst_nn.iloc[[9]], 2, return_distance=True)
#matches

known["Stockcode"].iloc[matches[0]]

myst['Stockcode'].iloc[[9]]
#Now, tokenise each descition.
#MAGIC TIME

myst.iloc[[9]].to_csv("myst.csv")
myst_nn.iloc[[9]].to_csv("myst_nn.csv")



known.iloc[matches[0]].to_csv("known.csv")
known_nn.iloc[matches[0]].to_csv("known_nn.csv")

results_wine = []

for index in range(len(myst.index)):
    distance, matches = nn_abs.kneighbors(myst_nn.iloc[[index]], 1, return_distance=True)
    results_wine.append(
        {
            'Mystery': "https://www.danmurphys.com.au/product/" + str(myst['Stockcode'].iloc[[index][0]]),
            'Matched': "https://www.danmurphys.com.au/product/" + str(known["Stockcode"].iloc[matches[0][0]]),
            'Distance': str(distance[0][0])
        }
    )


matched = pd.DataFrame(results_wine)

#def create_clickable_id(id):
#    url_template= '''<a href="../../link/to/{id}" target="_blank">{id}</a>'''.format(id=id)
#    return url_template

#matched['Mystery'] = matched['Mystery'].apply(create_clickable_id)
#matched['Matched'] = matched['Matched'].apply(create_clickable_id)

matched.to_html("matched.html")

matched.to_csv("matched_sat_1.csv")

matches = nn_abs.kneighbors(myst_nn.iloc[[3]], 2, return_distance=False)
known["Stockcode"].iloc[matches[0]]

myst['Stockcode'].iloc[[3]]




https://www.danmurphys.com.au/product/DM_MYSTERY352/under-wraps-beechworth-sangiovese-2018

https://www.danmurphys.com.au/product/7252


## xgboost

from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = known_nn
Y = known['Stockcode']

X_test = myst_nn
Y_test = myst['Stockcode']

# Dont need this if already cleaned ohe
#X.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X.columns.values]


#
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)


# fit model no training data
model = XGBClassifier(verbosity=2)
model.fit(X, Y)

# Check it out now
print(model)


### NB
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X, Y)
GaussianNB(priors=None)
y_pred = clf.predict(X_test_std)
## Now to predict

y_pred = clf.predict(X_test)
#y_pred holds the predicted label of your test set.
## Finally time to see the accuracy of our estimator.

from sklearn.metrics import accuracy_score
accuracy_score(y_true=y_test, y_pred=y_pred)

pd.DataFrame(y_pred).to_csv("Pred_NB.CSV")

pd.DataFrame(Y_test).to_csv("Myst_NB.CSV")Ansd t